In [1]:
import os
os.chdir("/content/drive/MyDrive/ug-project/src")
!pwd
%load_ext autoreload
%autoreload 2

/content/drive/MyDrive/ug-project/src


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
from datasets import FaceScrubDataset, FACESCRUB_MEAN, FACESCRUB_STD
import numpy as np
import torchvision.transforms.v2 as v2
from torchvision import datasets
from torch.utils.data import DataLoader, random_split

### Set hyperparameters

In [3]:
learning_rate = (0.05, 0.01, 0.005, 0.001)
epochs = 20
batch_size = 256

### Define transforms

In [4]:
prob = 0.2
color_transforms = [v2.RandomApply([v2.ColorJitter(brightness=0.2)], p=prob),
                    v2.RandomApply([v2.ColorJitter(hue=0.2)], p=prob),
                    v2.RandomApply([v2.ColorJitter(saturation=0.2)], p=prob),
                    v2.RandomApply([v2.ColorJitter(contrast=0.2)], p=prob),
                    v2.RandomEqualize(p=prob),
                    v2.RandomPosterize(bits=4, p=prob),
                    v2.RandomAutocontrast(p=prob)
                    ]
transform = v2.Compose([v2.ToPILImage(),
                        v2.Resize((256, 256)),
                        v2.RandomResizedCrop(size=224, scale=(0.5, 1.0), interpolation=v2.InterpolationMode.BICUBIC),
                        v2.RandomOrder(color_transforms),
                        v2.RandomHorizontalFlip(),
                        v2.ToTensor(),
                        v2.Normalize(mean=FACESCRUB_MEAN, std=FACESCRUB_STD),
                        v2.RandomErasing(p=0.5)
                        ])
transform_test = v2.Compose([
    v2.ToPILImage(),
    v2.Resize(224),
    v2.ToTensor(),
    v2.Normalize(mean=FACESCRUB_MEAN, std=FACESCRUB_STD)
])

/usr/local/lib/python3.11/dist-packages/torchvision/transforms/v2/_deprecated.py:42: UserWarning: The transform `ToTensor()` is deprecated and will be removed in a future release. Instead, please use `v2.Compose([v2.ToImage(), v2.ToDtype(torch.float32, scale=True)])`.Output is equivalent up to float precision.
  warnings.warn(


### Load FaceScrub dataset

In [5]:
train_dataset = FaceScrubDataset(train=True, hypertune=True, transform=transform)
print(f"Dev set size: {len(train_dataset)}")

val_dataset = FaceScrubDataset(train=False, hypertune=True, transform=transform_test)
print(f"Val set size: {len(val_dataset)}")

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2, pin_memory=True, drop_last=True, persistent_workers=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True, drop_last=True, persistent_workers=True)

Dev set size: 34517
Val set size: 4314


### Create ResNet18 model

In [6]:
import torch
import torch.nn as nn
from model import ResNet18Sup

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

criterion = nn.CrossEntropyLoss()
steps = len(train_loader)

### Loop through learning rates

In [7]:
for lr in learning_rate:
    print(f"Learning rate: {lr}")
    model = ResNet18Sup(num_classes=530).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=1e-5)
    warmup_scheduler = torch.optim.lr_scheduler.LinearLR(optimizer=optimizer, start_factor=0.01, end_factor=1.0,
                                                          total_iters=2*steps)
    decay_scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=optimizer, T_max=(epochs - 2) * steps)
    combined_scheduler = torch.optim.lr_scheduler.SequentialLR(optimizer=optimizer,
                                                                schedulers=[warmup_scheduler, decay_scheduler],
                                                                milestones=[2*steps+1])

    # Initialize metrics for ploting
    train_losses, train_correct = [], []
    val_losses, val_correct = [], []

    for epoch in range(0, epochs):

        ###################### Train model #########################
        model.train()
        for _, images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            logits = model(images)
            loss = criterion(logits, labels)
            loss.backward()
            optimizer.step()
            combined_scheduler.step()

        # Evaluate on training set
        model.eval()
        train_loss, train_corr, total_train = 0, 0, 0

        with torch.no_grad():
            for _, images, labels in train_loader:
                images, labels = images.to(device), labels.to(device)

                y_pred = model(images)
                loss = criterion(y_pred, labels)

                train_loss += loss.item() * images.size(0)
                predicted = torch.max(y_pred.data, 1)[1]
                train_corr += (predicted == labels).sum().item()
                total_train += labels.size(0)

        avg_train_loss = train_loss / total_train
        train_accuracy = (train_corr / total_train) * 100

        ###################### Evaluate model ######################
        val_loss, val_corr, total_val = 0, 0, 0

        with torch.no_grad():
            for _, images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)

                y_val_pred = model(images)
                loss = criterion(y_val_pred, labels)

                val_loss += loss.item() * images.size(0)
                predicted = torch.max(y_val_pred, 1)[1]
                val_corr += (predicted == labels).sum().item()
                total_val += labels.size(0)


        avg_val_loss = val_loss / total_val
        val_accuracy = (val_corr / total_val) * 100

        # Save results of current epoch
        train_losses.append(avg_train_loss)
        train_correct.append(train_accuracy)
        val_losses.append(avg_val_loss)
        val_correct.append(val_accuracy)

        # Add epoch results to log file
        log_entry = (f"Epoch {epoch+1}/{epochs}, Train Loss: {avg_train_loss:.4f}, Train Acc: {train_accuracy:.2f}, Val Loss: {avg_val_loss:.4f}, Val Acc: {val_accuracy:.2f}%")
        print(log_entry)

Learning rate: 0.05
Epoch 1/20, Train Loss: 6.2205, Train Acc: 0.62, Val Loss: 6.6778, Val Acc: 0.32%
Epoch 2/20, Train Loss: 6.1615, Train Acc: 1.70, Val Loss: 8.6967, Val Acc: 0.61%


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:240: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://github.com/pytorch/pytorch/issues/new/choose.
  warnings.warn(EPOCH_DEPRECATION_WARNING, UserWarning)


Epoch 3/20, Train Loss: 5.2474, Train Acc: 3.62, Val Loss: 7.5430, Val Acc: 1.10%
Epoch 4/20, Train Loss: 4.6657, Train Acc: 7.82, Val Loss: 8.5054, Val Acc: 2.34%
Epoch 5/20, Train Loss: 4.1303, Train Acc: 14.14, Val Loss: 8.5244, Val Acc: 3.98%
Epoch 6/20, Train Loss: 3.8894, Train Acc: 18.50, Val Loss: 10.4373, Val Acc: 4.98%
Epoch 7/20, Train Loss: 4.7468, Train Acc: 12.72, Val Loss: 12.0594, Val Acc: 3.22%
Epoch 8/20, Train Loss: 3.3584, Train Acc: 26.03, Val Loss: 10.3480, Val Acc: 7.03%
Epoch 9/20, Train Loss: 2.9768, Train Acc: 33.48, Val Loss: 11.9034, Val Acc: 8.67%
Epoch 10/20, Train Loss: 2.6283, Train Acc: 39.29, Val Loss: 12.1204, Val Acc: 9.45%
Epoch 11/20, Train Loss: 2.7705, Train Acc: 37.69, Val Loss: 12.8366, Val Acc: 9.59%
Epoch 12/20, Train Loss: 2.0486, Train Acc: 51.67, Val Loss: 13.7111, Val Acc: 12.35%
Epoch 13/20, Train Loss: 1.6648, Train Acc: 59.71, Val Loss: 13.9928, Val Acc: 13.96%
Epoch 14/20, Train Loss: 1.4087, Train Acc: 65.09, Val Loss: 15.3248, Val A